In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

In [ ]:
#new
def learnClasses1(train_ds,val_ds,nEpochs=32,cw = {}):
  
   earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 20, verbose = 1,restore_best_weights = True)
   
  
   inputs = tf.keras.Input(shape = (img_height,img_width,1))
   x = data_augmentation(inputs)
   x = tf.keras.layers.Rescaling(1.0 / 255)(x)
   x= tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
   x = tf.keras.layers.MaxPooling2D()(x)
   x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
   x = tf.keras.layers.MaxPooling2D()(x)
   x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
   x =  tf.keras.layers.MaxPooling2D()(x)
   x = tf.keras.layers.Flatten()(x)
   x = tf.keras.layers.Dense(128, activation='relu')(x)
   outputs = tf.keras.layers.Dense(num_classes,activation="softmax")(x)
  
   model = tf.keras.Model(inputs=inputs,outputs = outputs)
  
   optimizer = 'adam'
  
   # model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
   model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])
   model.summary()
   history = model.fit(train_ds, epochs=nEpochs, validation_data=val_ds, verbose=1,class_weight=cw,callbacks=[earlystop])

   hd = history.history
   lv = hd['loss']
   lv1 = hd['val_loss']
   acc = hd['accuracy']
   acc1 = hd['val_accuracy']
   epochs = range(1,len(lv)+1)
   plt.figure(1)
   plt.clf()
   plt.subplot(2,1,2)
   plt.plot(epochs,acc,'o-')
   plt.plot(epochs,acc1,'o-')
   plt.subplot(2,1,1)
   plt.plot(epochs,lv,'o-')
   plt.plot(epochs,lv1,'o-')
   return model

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
    ]
)

In [ ]:
import pathlib
#dataset_url = ".keras\\datasets\\types recepteurs"
data_dir = ".keras\\datasets\\types recepteurs"
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*\*.png')))
#print(data_dir)
print(image_count)

In [ ]:
batch_size = 32
img_height = 60
img_width = 60

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  color_mode="grayscale",
  label_mode="int",
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  color_mode="grayscale",
  label_mode="int",
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
#new
num_classes = len(class_names)
model=learnClasses1(train_ds,val_ds,nEpochs=32,cw = {})

In [ ]:
rootdir = "data"
liste = []
for subdir, dirs, files in os.walk(rootdir):    
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".png"):            
            #print (filepath)
            liste.append(filepath)
print(liste)
          

In [ ]:
#new
def getLabels(trainedModel, nameList, color_mode='grayscale'):
    ll=[]
    for nn in nameList:
        nnn=nn
        ii=tf.keras.utils.load_img(nnn,color_mode=color_mode, target_size=(img_width, img_height))
        ll.append(ii)
    label_list=[]
    for im in ll:
        tt=tf.keras.preprocessing.image.img_to_array(im)
        tt=tf.expand_dims(tt,0)
        rr=trainedModel.predict(tt)
        ind=rr[0].argmax()
        label_list.append(ind)
    return label_list

In [ ]:
list=getLabels(model, liste[0:100000])

In [ ]:
list